In [1]:
import pandas as pd
import os

import re 
import numpy as np
import cv2
from matplotlib.pyplot import imread
from keras.utils import np_utils
import matplotlib.pyplot as plt

from zipfile import ZipFile
import shutil
cwd = os.getcwd()
cwd

'/home/tf/HAR-UP/dataset'

In [2]:
def handle_name(path_name) :
    img_name = []
    for path in path_name :
        if len(path) == 68: 
            img_name.append(path[38:64])
        elif len(path) == 69 :
            img_name.append(path[39:65])
        else :
            img_name.append(path[40:66])
    handle = []
    for name in img_name :
        n1 = 13
        a1 = name.replace(name[n1],':')
        n2 = 16
        a2 = a1.replace(name[n2],':')
        handle.append(a2)
    return handle 

In [3]:
def ShowImage(ImageList, nRows = 1, nCols = 2, WidthSpace = 0.00, HeightSpace = 0.00):
    from matplotlib import pyplot as plt 
    import matplotlib.gridspec as gridspec
    gs = gridspec.GridSpec(nRows, nCols)     
    gs.update(wspace=WidthSpace, hspace=HeightSpace) # set the spacing between axes.
    plt.figure(figsize=(20,20))
    for i in range(len(ImageList)):
        ax1 = plt.subplot(gs[i])
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
        ax1.set_aspect('equal')

        plt.subplot(nRows, nCols,i+1)

        image = ImageList[i].copy()
        if (len(image.shape) < 3):
            plt.imshow(image, plt.cm.gray)
        else:
            plt.imshow(image)
        plt.title("Image " + str(i))
        plt.axis('off')

    plt.show()
    
    
def ResizeImage(IM, DesiredWidth, DesiredHeight):
    OrigWidth = float(IM.shape[1])
    OrigHeight = float(IM.shape[0])
    Width = DesiredWidth 
    Height = DesiredHeight

    if((Width == 0) & (Height == 0)):
        return IM
    
    if(Width == 0):
        Width = int((OrigWidth * Height)/OrigHeight)

    if(Height == 0):
        Height = int((OrigHeight * Width)/OrigWidth)

    dim = (Width, Height)
    resizedIM = cv2.resize(IM, dim, interpolation = cv2.INTER_NEAREST) 
    return resizedIM

In [4]:
def Extract_TimeLabel() :
    data = []
    count = 0 
    for sub_ in range(1,17 + 1 ) :
        Sub = 'Subject' + str(sub_)
        for act_ in range(1,11+1):
            Act = 'Activity'+ str(act_)
            for trial_ in range(1,3+1):
                Trial = 'Trial'+ str(trial_)
                if ( (sub_ == 8 and act_ == 11) and ( trial_ == 2 or trial_ == 3) ) :
                    print('----------------------------NULL---------------------------------')
                    continue 
                else :
                    path = Sub + '/' + Act + '/' + Trial + '/' + Sub + Act + Trial + '.csv'

                csv = pd.read_csv(path,skiprows=1)
                count+= 1
                data.append(csv)
                #if count % 100 == 0 :
                print('path : {} . Shape : ({},{})'.format(path, csv.shape[0], csv.shape[1]))
    final = pd.concat(data)
    index = final.index
    date =[]
    for i in index :
        date.append(i[0])        
    label = final.iloc[:, -1].values
    Time_Label = pd.DataFrame(label , index = date)
    return Time_Label

In [5]:
Time_Label = Extract_TimeLabel()

path : Subject1/Activity1/Trial1/Subject1Activity1Trial1.csv . Shape : (195,43)
path : Subject1/Activity1/Trial2/Subject1Activity1Trial2.csv . Shape : (193,43)
path : Subject1/Activity1/Trial3/Subject1Activity1Trial3.csv . Shape : (194,43)
path : Subject1/Activity2/Trial1/Subject1Activity2Trial1.csv . Shape : (157,43)
path : Subject1/Activity2/Trial2/Subject1Activity2Trial2.csv . Shape : (195,43)
path : Subject1/Activity2/Trial3/Subject1Activity2Trial3.csv . Shape : (194,43)
path : Subject1/Activity3/Trial1/Subject1Activity3Trial1.csv . Shape : (192,43)
path : Subject1/Activity3/Trial2/Subject1Activity3Trial2.csv . Shape : (191,43)
path : Subject1/Activity3/Trial3/Subject1Activity3Trial3.csv . Shape : (190,43)
path : Subject1/Activity4/Trial1/Subject1Activity4Trial1.csv . Shape : (189,43)
path : Subject1/Activity4/Trial2/Subject1Activity4Trial2.csv . Shape : (191,43)
path : Subject1/Activity4/Trial3/Subject1Activity4Trial3.csv . Shape : (193,43)
path : Subject1/Activity5/Trial1/Subject

In [7]:
os.getcwd()

'/home/tf/HAR-UP/dataset'

In [8]:
nan = np.load('Background_rm/3D data/Nan_index.npy',allow_pickle = True)
set_nan = set(nan)
len(set_nan)

36565

In [19]:
def imageName(start_sub, end_sub ,start_act  , end_act ,  start_cam ,  end_cam , DesiredWidth = 32, DesiredHeight = 32 ):
    name_img = []
    all_file_names = []
    for sub_ in range(start_sub, end_sub+1):
        sub = 'Subject' + str(sub_)

        for act_ in range(start_act, end_act + 1) :
            act = 'Activity' + str(act_)

            for trial_ in range(1, 3 + 1):

                trial = 'Trial'+ str(trial_)
                if ( (sub_ == 8 and act_ == 11) and ( trial_ == 2 or trial_ == 3) ) :
                    print('----------------------------NULL---------------------------------')
                    continue 
                for cam_ in range(start_cam , end_cam + 1) :
                    cam = 'Camera' + str(cam_)
                    count = 0
                    numFiles = 0
                    
                    dir_file = sub + '/' + act  + '/'+ trial +  '/' + sub + act  + trial + cam 
                    dir_ = 'CAMERA/' + sub + act  + trial  + cam
                    with ZipFile( dir_file + '.zip', 'r') as zipObj:
                        zipObj.extractall(dir_)
                    filenames = os.listdir(dir_)
                    filenames.sort()
                    all_file_names.append(filenames)
                    for filename in filenames:
                        
                        if filename == 'Subject4Activity1Trial1Camera2Synced':
                            continue
                        if filename in set_nan :
#                             print('Remove Ind xxxxxxxxxxxxxxxxxxx ')
                            continue
# ---------------------------------------------------------------------------------------------------
                        filepath = os.path.join(dir_, filename)
                        
                        if filepath == 'CAMERA/Subject6Activity10Trial2Camera2/2018-07-06T12_03_04.483526.png'  :
                            print('----------------------------NO SHAPE---------------------------------')
                            continue
                        if len(filepath) > 70 :
                            print(' {} : Invalid image'.format(filepath))
                            continue 
                        
                        
                        count += 1 
                        numFiles = numFiles + 1
#                         filepath = os.path.join(dir_, filename)
                        
                        if (count % 36 == 18):
                            filePath1 = filepath
                        if (count % 36 == 21):
                            filePath2 = filepath
                        if (count % 36 == 24):
                            filePath3 = filepath
                        if (count % 36 == 27):
                            filePath4 = filepath
                        if (count % 36 == 30):
                            filePath5 = filepath
                        if (count % 36 == 33):
                            filePath6 = filepath
                        if (count % 36 == 0):
                            filePath7 = filepath
                        if (count % 36 == 3):
                            filePath8 = filepath
                        if (count % 36 == 6):
                            filePath9 = filepath
                        if (count % 36 == 9):
                            filePath10 = filepath
                        if (count % 36 == 12):
                            filePath11 = filepath
                        if (count % 36 == 15):
                            filePath12 = filepath
                            
                        if (count % 36 == 0):
                            name_img.append(filePath1)
                            #print(filePath1)
                        if (count % 36 == 3 and count > 3):
                            name_img.append(filePath2)
                            #print(filePath2)
                        if (count % 36 == 6 and count > 6):
                            name_img.append(filePath3)
                            #print(filePath3)
                        if (count % 36 == 9 and count > 9):
                            name_img.append(filePath4)
                            #print(filePath4)
                        if (count % 36 == 12 and count > 12):
                            name_img.append(filePath5)
                            #print(filePath5)
                        if (count % 36 == 15 and count > 15):
                            name_img.append(filePath6)
                            #print(filePath6)
                        if (count % 36 == 18 and count > 18):
                            name_img.append(filePath7)
                        if (count % 36 == 21 and count > 21):
                            name_img.append(filePath8)
                        if (count % 36 == 24 and count > 24):
                            name_img.append(filePath9)
                        if (count % 36 == 27 and count > 27):
                            name_img.append(filePath10)
                        if (count % 36 == 30 and count > 30):
                            name_img.append(filePath11)
                        if (count % 36 == 33 and count > 33):
                            name_img.append(filePath12)
                        
                    print(sub + act + trial + cam)
                    print('Num images : {}, num clip : {}'.format(numFiles, len(name_img)))
                    shutil.rmtree(dir_)
                    
    #print(count)
    return name_img, all_file_names

def handle_name(name) :
    for i in range(len(name)):
        name[i] = name[i][-30:-4]
        name[i] = name[i].replace(name[i][13],':')
        name[i] = name[i].replace(name[i][16],':')
    label = Time_Label.loc[name].values.ravel()
    
    return label

In [9]:
def imageName(start_sub, end_sub ,start_act  , end_act ,  start_cam ,  end_cam , DesiredWidth = 32, DesiredHeight = 32 ):
    name_img = []
    for sub_ in range(start_sub, end_sub+1):
        sub = 'Subject' + str(sub_)

        for act_ in range(start_act, end_act + 1) :
            act = 'Activity' + str(act_)

            for trial_ in range(1, 3 + 1):

                trial = 'Trial'+ str(trial_)
                if ( (sub_ == 8 and act_ == 11) and ( trial_ == 2 or trial_ == 3) ) :
                    print('----------------------------NULL---------------------------------')
                    continue 
                for cam_ in range(start_cam , end_cam + 1) :
                    
                    
                    count = 0
                    
                    cam = 'Camera'+ str(cam_)
                    cwd = os.getcwd()
                    dir = 'CAMERA/' + sub + act + trial + cam
                    filenames = os.listdir(dir)
                    filenames.sort()
                    numFiles = 0
                    
                    
                    
                    count = 0
                    cam = 'Camera'+ str(cam_)
                    dir_ = 'CAMERA/' + sub + act + trial + cam
                    filenames = os.listdir(dir)
                    filenames.sort()
                    numFiles = 0
                    for filename in filenames:
                        count += 1 
                        if filename == 'Subject4Activity1Trial1Camera2Synced':
                            continue
                        if filename in set_nan :
                            print('Remove Ind xxxxxxxxxxxxxxxxxxx ')
                            continue
# ---------------------------------------------------------------------------------------------------
                        filepath = os.path.join(dir_, filename)
                        
                        if filepath == 'CAMERA/Subject6Activity10Trial2Camera2/2018-07-06T12_03_04.483526.png'  :
                            print('----------------------------NO SHAPE---------------------------------')
                            continue
                        if len(filepath) > 70 :
                            print(' {} : Invalid image'.format(filepath))
                            continue 
                        
                        filepath = os.path.join(dir, filename)
                        numFiles = numFiles + 1
                        
                        if (count % 36 == 18):
                            filePath1 = filepath
                        if (count % 36 == 21):
                            filePath2 = filepath
                        if (count % 36 == 24):
                            filePath3 = filepath
                        if (count % 36 == 27):
                            filePath4 = filepath
                        if (count % 36 == 30):
                            filePath5 = filepath
                        if (count % 36 == 33):
                            filePath6 = filepath
                        if (count % 36 == 0):
                            filePath7 = filepath
                        if (count % 36 == 3):
                            filePath8 = filepath
                        if (count % 36 == 6):
                            filePath9 = filepath
                        if (count % 36 == 9):
                            filePath10 = filepath
                        if (count % 36 == 12):
                            filePath11 = filepath
                        if (count % 36 == 15):
                            filePath12 = filepath
                            
                        if (count % 36 == 0):
                            name_img.append(filePath1)
                            #print(filePath1)
                        if (count % 36 == 3 and count > 3):
                            name_img.append(filePath2)
                            #print(filePath2)
                        if (count % 36 == 6 and count > 6):
                            name_img.append(filePath3)
                            #print(filePath3)
                        if (count % 36 == 9 and count > 9):
                            name_img.append(filePath4)
                            #print(filePath4)
                        if (count % 36 == 12 and count > 12):
                            name_img.append(filePath5)
                            #print(filePath5)
                        if (count % 36 == 15 and count > 15):
                            name_img.append(filePath6)
                            #print(filePath6)
                        if (count % 36 == 18 and count > 18):
                            name_img.append(filePath7)
                        if (count % 36 == 21 and count > 21):
                            name_img.append(filePath8)
                        if (count % 36 == 24 and count > 24):
                            name_img.append(filePath9)
                        if (count % 36 == 27 and count > 27):
                            name_img.append(filePath10)
                        if (count % 36 == 30 and count > 30):
                            name_img.append(filePath11)
                        if (count % 36 == 33 and count > 33):
                            name_img.append(filePath12)
                        
                    print(sub + act + trial + cam)
                    print(numFiles)
                    
    #print(count)
    return name_img

def handle_name(name) :
    for i in range(len(name)):
        name[i] = name[i][-30:-4]
        name[i] = name[i].replace(name[i][13],':')
        name[i] = name[i].replace(name[i][16],':')
    label = Time_Label.loc[name].values.ravel()
    
    return label

In [10]:
def IMG(start_sub, end_sub ,start_act  , end_act ,  start_cam ,  end_cam , DesiredWidth = 32, DesiredHeight = 32 ):
    IMG = []
    num_img = []
    for sub_ in range(start_sub, end_sub+1):
        sub = 'Subject' + str(sub_)

        for act_ in range(start_act, end_act + 1) :
            act = 'Activity' + str(act_)

            for trial_ in range(1, 3 + 1):
                trial = 'Trial'+ str(trial_)
                if ( (sub_ == 8 and act_ == 11) and ( trial_ == 2 or trial_ == 3) ) :
                    print('----------------------------NULL---------------------------------')
                    continue 
                for cam_ in range(start_cam , end_cam + 1) :
                    
                    cam = 'Camera'+ str(cam_)
#                     ----------------------
                    clip1 = []
                    clip2 = []
                    clip3 = []
                    clip4 = []
                    clip5 = []
                    clip6 = []
                    clip7 = []
                    clip8 = []
                    clip9 = []
                    clip10 = []
                    clip11 = []
                    clip12 = []
#                     ----------------------                 
                    count = 0
                    numFiles = 0
                    
                    dir_file = sub + '/' + act  + '/'+ trial +  '/' + sub + act  + trial + cam 
                    dir_ = 'CAMERA/' + sub + act  + trial  + cam
                    with ZipFile( dir_file + '.zip', 'r') as zipObj:
                        zipObj.extractall(dir_)
                    filenames = os.listdir(dir_)
                    filenames.sort()
                    for filename in filenames:
#                         print(filename)
                        if filename == 'Subject4Activity1Trial1Camera2Synced':
                            continue
                        if filename in set_nan :
#                             print('Remove Ind xxxxxxxxxxxxxxxxxxx ')
                            continue
# ---------------------------------------------------------------------------------------------------
                        filepath = os.path.join(dir_, filename)
                        
                        if filepath == 'CAMERA/Subject6Activity10Trial2Camera2/2018-07-06T12_03_04.483526.png'  :
                            print('----------------------------NO SHAPE---------------------------------')
                            continue
                        if len(filepath) > 70 :
                            print(' {} : Invalid image'.format(filepath))
                            continue 
        
        
        
                        count += 1 
                        filepath = os.path.join(dir_, filename)
                        numFiles = numFiles + 1
                                
                        #print(filepath)
                        img = cv2.imread(filepath, 0) 
                        resized = ResizeImage(img, DesiredWidth, DesiredHeight)
                        
                        clip1.append(resized)
                        if (count > 3):
                            clip2.append(resized)
                        if (count > 6):
                            clip3.append(resized)
                        if (count > 9):
                            clip4.append(resized)
                        if (count > 12):
                            clip5.append(resized)
                        if (count > 15):
                            clip6.append(resized)
                        if (count > 18):
                            clip7.append(resized)
                        if (count > 21):
                            clip8.append(resized)
                        if (count > 24):
                            clip9.append(resized)
                        if (count > 27):
                            clip10.append(resized)
                        if (count > 30):
                            clip11.append(resized)
                        if (count > 33):
                            clip12.append(resized)
                            
                        if (count % 36 == 0):
                            IMG.append(clip1)
                            clip1 = []
                        if (count % 36 == 3 and count > 3):
                            IMG.append(clip2)
                            clip2 = []
                        if (count % 36 == 6 and count > 6):
                            IMG.append(clip3)
                            clip3 = []
                        if (count % 36 == 9 and count > 9):
                            IMG.append(clip4)
                            clip4 = []
                        if (count % 36 == 12 and count > 12):
                            IMG.append(clip5)
                            clip5 = []
                        if (count % 36 == 15 and count > 15):
                            IMG.append(clip6)
                            clip6 = []
                        if (count % 36 == 18 and count > 18):
                            IMG.append(clip7)
                            clip7 = []
                        if (count % 36 == 21 and count > 21):
                            IMG.append(clip8)
                            clip8 = []
                        if (count % 36 == 24 and count > 24):
                            IMG.append(clip9)
                            clip9 = []    
                        if (count % 36 == 27 and count > 27):
                            IMG.append(clip10)
                            clip10 = []
                        if (count % 36 == 30 and count > 30):
                            IMG.append(clip11)
                            clip11 = []
                        if (count % 36 == 33 and count > 33):
                            IMG.append(clip12)
                            clip12 = []  
                        
                    print(sub + act + trial + cam)
                    print('Num images : {}, num clip : {}'.format(numFiles, len(IMG)))
                    num_img.append(numFiles)
                    shutil.rmtree(dir_)
    #print(count)
    return IMG,num_img

# Video CAM 1 

## Video

In [11]:
startSubject = 1
endSubject = 3           # set this value to 17 for all dataset 
startActivity = 1
endActivity = 11
startCamera = 1
endCamera = 1
img_1, num_img_1 = IMG(startSubject, 
                       endSubject, 
                       startActivity, 
                       endActivity, 
                       startCamera, 
                       endCamera)


Subject1Activity1Trial1Camera1
Num images : 195, num clip : 54
Subject1Activity1Trial2Camera1
Num images : 193, num clip : 107
Subject1Activity1Trial3Camera1
Num images : 194, num clip : 160
Subject1Activity2Trial1Camera1
Num images : 157, num clip : 201
Subject1Activity2Trial2Camera1
Num images : 195, num clip : 255
Subject1Activity2Trial3Camera1
Num images : 194, num clip : 308
Subject1Activity3Trial1Camera1
Num images : 192, num clip : 361
Subject1Activity3Trial2Camera1
Num images : 191, num clip : 413
Subject1Activity3Trial3Camera1
Num images : 190, num clip : 465
Subject1Activity4Trial1Camera1
Num images : 189, num clip : 517
Subject1Activity4Trial2Camera1
Num images : 191, num clip : 569
Subject1Activity4Trial3Camera1
Num images : 193, num clip : 622
Subject1Activity5Trial1Camera1
Num images : 189, num clip : 674
Subject1Activity5Trial2Camera1
Num images : 182, num clip : 723
Subject1Activity5Trial3Camera1
Num images : 187, num clip : 774
Subject1Activity6Trial1Camera1
Num images

In [12]:
np.array(img_1).shape


# actual size for 17 subject is : (80491, 36, 32, 32)

(16379, 36, 32, 32)

In [13]:
sum(num_img_1)

# actual img for 17 subject is : 258113

52500

In [166]:
# SAVE DATA 

# path = 'Background_rm/3D data/image_1_3D_36.npy'
# np.save(path,np.array(img_1))

## Label

In [20]:
label_1, all_files_1 = imageName(start_sub = 1,
                                end_sub = 3,    # set this value to 17 for all dataset 
                                start_act = 1,
                                end_act = 11,
                                start_cam = 1,
                                end_cam = 1)

Subject1Activity1Trial1Camera1
Num images : 195, num clip : 54
Subject1Activity1Trial2Camera1
Num images : 193, num clip : 107
Subject1Activity1Trial3Camera1
Num images : 194, num clip : 160
Subject1Activity2Trial1Camera1
Num images : 157, num clip : 201
Subject1Activity2Trial2Camera1
Num images : 195, num clip : 255
Subject1Activity2Trial3Camera1
Num images : 194, num clip : 308
Subject1Activity3Trial1Camera1
Num images : 192, num clip : 361
Subject1Activity3Trial2Camera1
Num images : 191, num clip : 413
Subject1Activity3Trial3Camera1
Num images : 190, num clip : 465
Subject1Activity4Trial1Camera1
Num images : 189, num clip : 517
Subject1Activity4Trial2Camera1
Num images : 191, num clip : 569
Subject1Activity4Trial3Camera1
Num images : 193, num clip : 622
Subject1Activity5Trial1Camera1
Num images : 189, num clip : 674
Subject1Activity5Trial2Camera1
Num images : 182, num clip : 723
Subject1Activity5Trial3Camera1
Num images : 187, num clip : 774
Subject1Activity6Trial1Camera1
Num images

In [ ]:
# SAVE LABEL 

# label_1s = handle_name(label_1)
# path = 'Background_rm/3D data/label_1_3D_36.npy'
# np.save(path,np.array(label_1s))

In [ ]:
del img_1 , num_img_1

# Video CAM 2

## Video

In [14]:
startSubject = 1
endSubject = 3        # set this value to 17 for all dataset 
startActivity = 1
endActivity = 11
startCamera = 2
endCamera = 2
img_2, num_img_2 = IMG(startSubject, endSubject, startActivity, endActivity, startCamera, endCamera)


Subject1Activity1Trial1Camera2
Num images : 195, num clip : 54
Subject1Activity1Trial2Camera2
Num images : 193, num clip : 107
Subject1Activity1Trial3Camera2
Num images : 194, num clip : 160
Subject1Activity2Trial1Camera2
Num images : 157, num clip : 201
Subject1Activity2Trial2Camera2
Num images : 195, num clip : 255
Subject1Activity2Trial3Camera2
Num images : 194, num clip : 308
Subject1Activity3Trial1Camera2
Num images : 192, num clip : 361
Subject1Activity3Trial2Camera2
Num images : 191, num clip : 413
Subject1Activity3Trial3Camera2
Num images : 190, num clip : 465
Subject1Activity4Trial1Camera2
Num images : 189, num clip : 517
Subject1Activity4Trial2Camera2
Num images : 191, num clip : 569
Subject1Activity4Trial3Camera2
Num images : 193, num clip : 622
Subject1Activity5Trial1Camera2
Num images : 189, num clip : 674
Subject1Activity5Trial2Camera2
Num images : 182, num clip : 723
Subject1Activity5Trial3Camera2
Num images : 187, num clip : 774
Subject1Activity6Trial1Camera2
Num images

In [15]:
np.array(img_2).shape

# actual size for 17 subject is : (80491, 36, 32, 32)

(16380, 36, 32, 32)

In [16]:
sum(num_img_2)

# actual img for 17 subject is : 258113

52501

## Label 

In [ ]:
label_2, all_files_2 = imageName(start_sub = 1,
                    end_sub = 3,   # set this value to 17 for all dataset 
                    start_act = 1,
                    end_act = 11,
                    start_cam = 2,
                    end_cam = 2)




Subject1Activity1Trial1Camera2
Num images : 195, num clip : 54
Subject1Activity1Trial2Camera2
Num images : 193, num clip : 107
Subject1Activity1Trial3Camera2
Num images : 194, num clip : 160
Subject1Activity2Trial1Camera2
Num images : 157, num clip : 201
Subject1Activity2Trial2Camera2
Num images : 195, num clip : 255
Subject1Activity2Trial3Camera2
Num images : 194, num clip : 308
Subject1Activity3Trial1Camera2
Num images : 192, num clip : 361
Subject1Activity3Trial2Camera2
Num images : 191, num clip : 413
Subject1Activity3Trial3Camera2
Num images : 190, num clip : 465
Subject1Activity4Trial1Camera2
Num images : 189, num clip : 517
Subject1Activity4Trial2Camera2
Num images : 191, num clip : 569
Subject1Activity4Trial3Camera2
Num images : 193, num clip : 622
Subject1Activity5Trial1Camera2
Num images : 189, num clip : 674
Subject1Activity5Trial2Camera2
Num images : 182, num clip : 723
Subject1Activity5Trial3Camera2
Num images : 187, num clip : 774
Subject1Activity6Trial1Camera2
Num images

In [ ]:
# SAVE LABEL

# label_2s = handle_name(label_2)
# path = 'Background_rm/3D data/label_2_3D_36.npy'
# np.save(path,np.array(label_2s)

In [ ]:
del img_2 , num_img_2